# Ternviz

Make a video of a turning structure from SMILES.

Checkout the Twitter bot [@ternviz](https://twitter.com/ternviz)!

In [ ]:
# @title Install Required Packages
print("🐍Installing Miniconda")
! wget -q https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local > /dev/null  && rm Miniconda3-py37_4.8.2-Linux-x86_64.sh
import sys, os

os.environ["CONDA_PREFIX"] = "/usr/local/"
sys.path.append("/usr/local/lib/python3.7/site-packages/")
print("📦Installing Conda Packages")
!conda install -c conda-forge -qy --no-deps vmd pthread-stubs open-fonts libgomp libgcc-ng > /dev/null
print("🪶Installing ternviz")
!pip install -q git+https://github.com/whitead/ternviz.git > /dev/null
print("🎉Done!")

In [ ]:
# @title Run Parameters
import ternviz
from IPython.core.display import Video, display
import os

smiles1 = "CCO"  # @param {type:"string"}
high_quality = False  # @param {type:"boolean"}
# @markdown ### Optional Parameters
# @markdown *Enter another SMILES here to compare two molecules*
smiles2 = ""  # @param {type:"string"}
# @markdown *Name your molecules*
name1 = ""  # @param {type:"string"}
name2 = ""  # @param {type:"string"}
names = []
if name1:
    names.append(name1)
if name2:
    names.append(name2)

split = False

smiles = [smiles1]
if smiles2:
    smiles = [smiles1, smiles2]
    split = True

if not names:
    names = [ternviz.get_name(s) for s in smiles]
width = 400 if split else 800
movies = []
for i, (n, s) in enumerate(zip(names, smiles)):
    print("😁 Processing SMILES", i + 1, "of", len(smiles))
    sm = ternviz.check_smiles(s)
    if sm:
        raise ValueError(sm)
    p = ternviz.gen_coords(s)
    if not p:
        raise ValueError("Failed to generate coordinates")
    print("🎨 Rendering...", s)
    ternviz.render(p.name, width, id=n, high_quality=high_quality)
    print("🎥 Making Movie for", s)
    m = ternviz.movie(n, short_name=n)
    movies.append(m)
    p.close()
    os.unlink(p.name)
    display(Video(m, embed=True))
if split:
    out = ternviz.multiplex(movies, "out")
else:
    out = movies[0]

In [ ]:
# @title Download Movie
from google.colab import files

files.download(out)